In [1]:
import pandas as pd
import numpy as np
import altair as alt
import datetime
from sklearn.linear_model import LinearRegression
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [2]:
file = 'cpi_index.csv'
dat = pd.read_csv(file, index_col=0)
dat.index = pd.to_datetime(dat.index)
dat = dat.sort_index()

In [3]:
dat['2016':]

,cpi,cpi_goods,cpi_services,retail_sales,avg_hourly_earnings,ECI,PCE_core
2016-01-01,236.916,144.725,305.803,447561.0,25.38,124.5,97.567
2016-02-01,237.111,145.856,306.932,451821.0,25.38,NaN,97.736
2016-03-01,238.132,146.367,307.703,450561.0,25.45,NaN,97.876
2016-04-01,239.261,146.565,308.619,452443.0,25.53,125.4,98.117
2016-05-01,240.229,146.230,309.770,454003.0,25.58,NaN,98.284
...,...,...,...,...,...,...,...
2023-09-01,307.789,167.141,401.234,705304.0,34.01,NaN,119.842
2023-10-01,307.671,166.759,402.549,703528.0,34.10,162.0,120.010
2023-11-01,307.051,165.367,404.143,703336.0,34.23,NaN,120.088
2023-12-01,306.746,164.590,405.338,706180.0,34.36,NaN,120.294


In [4]:
# drop na's and get log of each column
dat.drop('ECI', axis=1, inplace = True)
dat.dropna(inplace=True)
for col in dat.columns:
    dat['ln_'+col] = np.log(dat[col])


In [5]:
dat['ind'] = [x for x in range(len(dat))]
dat

,cpi,cpi_goods,cpi_services,retail_sales,avg_hourly_earnings,PCE_core,ln_cpi,ln_cpi_goods,ln_cpi_services,ln_retail_sales,ln_avg_hourly_earnings,ln_PCE_core,ind
2006-03-01,199.800,141.500,242.400,355665.0,20.05,83.509,5.297317,4.952300,5.490589,12.781745,2.998229,4.424954,0
2006-04-01,201.500,141.700,243.200,357423.0,20.15,83.764,5.305789,4.953712,5.493884,12.786675,3.003204,4.428003,1
2006-05-01,202.500,141.500,243.700,356704.0,20.13,83.974,5.310740,4.952300,5.495938,12.784662,3.002211,4.430507,2
2006-06-01,202.900,140.700,244.700,357879.0,20.23,84.186,5.312713,4.946630,5.500033,12.787950,3.007167,4.433029,3
2006-07-01,203.500,139.600,245.800,359006.0,20.29,84.268,5.315666,4.938781,5.504518,12.791094,3.010128,4.434002,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,307.026,168.029,399.219,699540.0,33.91,119.449,5.726932,5.124137,5.989510,13.458178,3.523710,4.782890,209
2023-09-01,307.789,167.141,401.234,705304.0,34.01,119.842,5.729414,5.118838,5.994545,13.466384,3.526655,4.786174,210
2023-10-01,307.671,166.759,402.549,703528.0,34.10,120.010,5.729031,5.116550,5.997817,13.463863,3.529297,4.787575,211
2023-11-01,307.051,165.367,404.143,703336.0,34.23,120.088,5.727014,5.108167,6.001769,13.463590,3.533102,4.788225,212


In [6]:
def make_reg_line(col_name):
    '''takes a vector in dat and creates regression line to extend'''
    X = dat['2010':'2019'].ind.values.reshape(-1, 1) 
    y = dat['2010':'2019'][col_name].values.reshape(-1, 1) 
    reg = LinearRegression().fit(X, y)
    dat['pred_'+col_name] = reg.predict(dat.ind.values.reshape(-1, 1))

In [7]:
for item in ['ln_cpi', 'ln_cpi_goods', 'ln_cpi_services','ln_retail_sales', 'ln_avg_hourly_earnings']:
    make_reg_line(item)

In [8]:
dat

,cpi,cpi_goods,cpi_services,retail_sales,avg_hourly_earnings,PCE_core,ln_cpi,ln_cpi_goods,ln_cpi_services,ln_retail_sales,ln_avg_hourly_earnings,ln_PCE_core,ind,pred_ln_cpi,pred_ln_cpi_goods,pred_ln_cpi_services,pred_ln_retail_sales,pred_ln_avg_hourly_earnings
2006-03-01,199.800,141.500,242.400,355665.0,20.05,83.509,5.297317,4.952300,5.490589,12.781745,2.998229,4.424954,0,5.327896,4.988944,5.472682,12.648661,3.008618
2006-04-01,201.500,141.700,243.200,357423.0,20.15,83.764,5.305789,4.953712,5.493884,12.786675,3.003204,4.428003,1,5.329219,4.988858,5.474835,12.651850,3.010573
2006-05-01,202.500,141.500,243.700,356704.0,20.13,83.974,5.310740,4.952300,5.495938,12.784662,3.002211,4.430507,2,5.330542,4.988773,5.476988,12.655039,3.012527
2006-06-01,202.900,140.700,244.700,357879.0,20.23,84.186,5.312713,4.946630,5.500033,12.787950,3.007167,4.433029,3,5.331865,4.988688,5.479140,12.658228,3.014482
2006-07-01,203.500,139.600,245.800,359006.0,20.29,84.268,5.315666,4.938781,5.504518,12.791094,3.010128,4.434002,4,5.333189,4.988602,5.481293,12.661417,3.016437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,307.026,168.029,399.219,699540.0,33.91,119.449,5.726932,5.124137,5.989510,13.458178,3.523710,4.782890,209,5.604411,4.971103,5.922603,13.315171,3.417147
2023-09-01,307.789,167.141,401.234,705304.0,34.01,119.842,5.729414,5.118838,5.994545,13.466384,3.526655,4.786174,210,5.605734,4.971018,5.924756,13.318360,3.419101
2023-10-01,307.671,166.759,402.549,703528.0,34.10,120.010,5.729031,5.116550,5.997817,13.463863,3.529297,4.787575,211,5.607057,4.970932,5.926908,13.321549,3.421056
2023-11-01,307.051,165.367,404.143,703336.0,34.23,120.088,5.727014,5.108167,6.001769,13.463590,3.533102,4.788225,212,5.608380,4.970847,5.929061,13.324738,3.423011


In [9]:
raw = alt.Chart(dat['2000':].reset_index(), title = 'CPI all items').mark_line().encode(
    x=alt.X('index:T', axis=alt.Axis(title='Date')),
    y=alt.Y('ln_cpi:Q', axis=alt.Axis(title='Log CPI'), scale=alt.Scale(zero=False))
)

pred = alt.Chart(dat['2000':].reset_index()).mark_line(color = 'red').encode(
    x=alt.X('index:T'),
    y=alt.Y('pred_ln_cpi:Q', scale=alt.Scale(zero=False))
)


(raw + pred).configure_title(color = '#1f4e79', fontSize = 16, font = 'Arial', fontWeight = 'bold')

alt.LayerChart(...)

In [10]:
raw = alt.Chart(dat['2010':].reset_index(), title = 'CPI Goods').mark_line().encode(
    x=alt.X('index:T', axis=alt.Axis(title='Date')),
    y=alt.Y('ln_cpi_goods:Q', axis=alt.Axis(title='Log CPI Goods'), scale=alt.Scale(zero=False))
)

pred = alt.Chart(dat['2010':].reset_index()).mark_line(color = 'red').encode(
    x=alt.X('index:T'),
    y=alt.Y('pred_ln_cpi_goods:Q', scale=alt.Scale(zero=False))
)



(raw + pred).configure_title(color = '#1f4e79', fontSize = 16, font = 'Arial', fontWeight = 'bold')

alt.LayerChart(...)

In [11]:
raw = alt.Chart(dat['2020':].reset_index(), title = 'CPI Services').mark_line().encode(
    x=alt.X('index:T', axis=alt.Axis(title='Date')),
    y=alt.Y('ln_cpi_services:Q', axis=alt.Axis(title='Log CPI Services'), scale=alt.Scale(zero=False))
)

pred = alt.Chart(dat['2020':].reset_index()).mark_line(color = 'red').encode(
    x=alt.X('index:T'),
    y=alt.Y('pred_ln_cpi_services:Q', scale=alt.Scale(zero=False))
)



(raw + pred).configure_title(color = '#1f4e79', fontSize = 16)

alt.LayerChart(...)

In [12]:
raw = alt.Chart(dat['2010':].reset_index(), title = 'Retail Sales').mark_line().encode(
    x=alt.X('index:T', axis=alt.Axis(title='Date')),
    y=alt.Y('ln_retail_sales:Q', axis=alt.Axis(title='Log Retail Sales'), scale=alt.Scale(zero=False))
)

pred = alt.Chart(dat['2010':].reset_index()).mark_line(color = 'red').encode(
    x=alt.X('index:T'),
    y=alt.Y('pred_ln_retail_sales:Q', scale=alt.Scale(zero=False))
)



(raw + pred).configure_title(color = '#1f4e79', fontSize = 16, font = 'Arial', fontWeight = 'bold')

alt.LayerChart(...)

In [13]:
raw = alt.Chart(dat['2010':].reset_index(), title = 'Average Hourly Earnings').mark_line().encode(
    x=alt.X('index:T', axis=alt.Axis(title='Date')),
    y=alt.Y('ln_avg_hourly_earnings:Q', axis=alt.Axis(title='Log Average Hourly Earnings'), scale=alt.Scale(zero=False))
)

pred = alt.Chart(dat['2010':].reset_index()).mark_line(color = 'red').encode(
    x=alt.X('index:T'),
    y=alt.Y('pred_ln_avg_hourly_earnings:Q', scale=alt.Scale(zero=False))
)



(raw + pred).configure_title(color = '#1f4e79', fontSize = 16, font = 'Arial', fontWeight = 'bold')

alt.LayerChart(...)

In [14]:
dat1 = pd.read_csv(file, index_col=0)
dat1.index = pd.to_datetime(dat1.index)
dat1 = dat1.sort_index()
dat1.drop('avg_hourly_earnings', axis = 1, inplace=True)
dat1.dropna(inplace=True)
for col in dat1.columns:
    dat1['ln_'+col] = np.log(dat1[col])

In [15]:
# make a larger time frame of cpi goods and average hourly earnings
dat1['ind'] = [x for x in range(len(dat1))]
X = dat1['2000':'2019'].ind.values.reshape(-1, 1) 
y = dat1['2000':'2019']['ln_cpi_goods'].values.reshape(-1, 1) 
reg = LinearRegression().fit(X, y)
dat1['longpred_ln_cpi_goods'] = reg.predict(dat1.ind.values.reshape(-1, 1))

In [16]:
X = dat1['2000':'2019'].ind.values.reshape(-1, 1) 
y = dat1['2000':'2019']['ln_cpi_services'].values.reshape(-1, 1) 
reg = LinearRegression().fit(X, y)
dat1['longpred_ln_cpi_services'] = reg.predict(dat1.ind.values.reshape(-1, 1))



In [17]:
raw = alt.Chart(dat1['2000':].reset_index(), title = 'CPI Goods').mark_line().encode(
    x=alt.X('index:T', axis=alt.Axis(title='Date')),
    y=alt.Y('ln_cpi_goods:Q', axis=alt.Axis(title='Log CPI Goods'), scale=alt.Scale(zero=False))
)

pred = alt.Chart(dat1['2000':].reset_index()).mark_line(color = 'red').encode(
    x=alt.X('index:T'),
    y=alt.Y('longpred_ln_cpi_goods:Q', scale=alt.Scale(zero=False))
)



(raw + pred).configure_title(color = '#1f4e79', fontSize = 16, font = 'Arial', fontWeight = 'bold')

alt.LayerChart(...)

In [18]:
raw = alt.Chart(dat1['2000':].reset_index(), title = 'CPI Services').mark_line().encode(
    x=alt.X('index:T', axis=alt.Axis(title='Date')),
    y=alt.Y('ln_cpi_services:Q', axis=alt.Axis(title='Log CPI Services'), scale=alt.Scale(zero=False))
)

pred = alt.Chart(dat1['2000':].reset_index()).mark_line(color = 'red').encode(
    x=alt.X('index:T'),
    y=alt.Y('longpred_ln_cpi_services:Q', scale=alt.Scale(zero=False))
)



(raw + pred).configure_title(color = '#1f4e79', fontSize = 16, font = 'Arial', fontWeight = 'bold')

alt.LayerChart(...)

In [19]:
dat1

,cpi,cpi_goods,cpi_services,retail_sales,ECI,PCE_core,ln_cpi,ln_cpi_goods,ln_cpi_services,ln_retail_sales,ln_ECI,ln_PCE_core,ind,longpred_ln_cpi_goods,longpred_ln_cpi_services
2001-01-01,175.100,144.800,205.700,278834.0,87.6,75.796,5.165357,4.975353,5.326419,12.538372,4.472781,4.328046,0,4.949247,5.353062
2001-04-01,176.900,146.600,208.000,280808.0,88.4,76.130,5.175585,4.987708,5.337538,12.545426,4.481872,4.332442,1,4.949706,5.359342
2001-07-01,177.500,144.400,210.100,279504.0,89.2,76.515,5.178971,4.972587,5.347584,12.540772,4.490881,4.337487,2,4.950166,5.365622
2001-10-01,177.700,145.600,211.700,294540.0,90.0,76.654,5.180097,4.980863,5.355170,12.593170,4.499810,4.339302,3,4.950625,5.371902
2002-01-01,177.100,143.700,213.800,283508.0,90.7,76.871,5.176715,4.967728,5.365041,12.554996,4.507557,4.342129,4,4.951084,5.378181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-01,298.012,166.601,381.580,688352.0,155.3,116.087,5.697134,5.115602,5.944321,13.442056,5.045359,4.754340,87,4.989216,5.899403
2023-01-01,299.170,165.340,387.258,695776.0,157.1,117.461,5.701012,5.108004,5.959091,13.452783,5.056883,4.766106,88,4.989675,5.905683
2023-04-01,303.363,167.833,393.769,684636.0,158.6,118.642,5.714930,5.122969,5.975764,13.436643,5.066385,4.776111,89,4.990135,5.911963
2023-07-01,305.691,168.014,398.002,694415.0,160.5,119.332,5.722575,5.124047,5.986457,13.450825,5.078294,4.781910,90,4.990594,5.918243


In [20]:
def delta(vector, window):
    ''' takes a vector and a window and returns a
        a vector that is the change over a certain
        number (window) of rows'''
    chg = []
    for h in range(0,(window-1)):
        chg.append(np.nan)
    for i in range(len(vector)-(window-1)):
        chg.append(((vector[i+(window-1)] - vector[i])/vector[i]))
    
    return chg

In [21]:
df2 = pd.DataFrame()
for col in ['cpi', 'cpi_goods', 'cpi_services']:
    df2[col] = delta(dat[col], 13)
    
df2['dates'] = dat.index

In [22]:
df2

,cpi,cpi_goods,cpi_services,dates
0,NaN,NaN,NaN,2006-03-01
1,NaN,NaN,NaN,2006-04-01
2,NaN,NaN,NaN,2006-05-01
3,NaN,NaN,NaN,2006-06-01
4,NaN,NaN,NaN,2006-07-01
...,...,...,...,...
209,0.036651,0.002338,0.058993,2023-08-01
210,0.036997,0.000221,0.056897,2023-09-01
211,0.032411,0.000948,0.054953,2023-10-01
212,0.031373,0.000460,0.054711,2023-11-01


In [23]:

cpidetails = df2.melt(id_vars = "dates", value_vars = ['cpi', 'cpi_goods', 'cpi_services'])
cpidetails

,dates,variable,value
0,2006-03-01,cpi,NaN
1,2006-04-01,cpi,NaN
2,2006-05-01,cpi,NaN
3,2006-06-01,cpi,NaN
4,2006-07-01,cpi,NaN
...,...,...,...
637,2023-08-01,cpi_services,0.058993
638,2023-09-01,cpi_services,0.056897
639,2023-10-01,cpi_services,0.054953
640,2023-11-01,cpi_services,0.054711


In [24]:
df2

,cpi,cpi_goods,cpi_services,dates
0,NaN,NaN,NaN,2006-03-01
1,NaN,NaN,NaN,2006-04-01
2,NaN,NaN,NaN,2006-05-01
3,NaN,NaN,NaN,2006-06-01
4,NaN,NaN,NaN,2006-07-01
...,...,...,...,...
209,0.036651,0.002338,0.058993,2023-08-01
210,0.036997,0.000221,0.056897,2023-09-01
211,0.032411,0.000948,0.054953,2023-10-01
212,0.031373,0.000460,0.054711,2023-11-01


In [25]:
main = alt.Chart(cpidetails).mark_line().encode(
    x=alt.X('dates:T', axis = alt.Axis(title = 'date')),
    y=alt.Y('value', axis=alt.Axis(format='%', title='percentage change YoY'))

)

rule = alt.Chart(cpidetails).mark_rule(color='red').encode(
    y='mean(value):Q'
)

alt.layer(
    main, rule,
    data=cpidetails
).facet(
    column=alt.Column('variable:N', header=alt.Header(title='CPI Components'))
).resolve_axis(
    x='independent',
    y='independent',
)

alt.FacetChart(...)

In [26]:
dat

,cpi,cpi_goods,cpi_services,retail_sales,avg_hourly_earnings,PCE_core,ln_cpi,ln_cpi_goods,ln_cpi_services,ln_retail_sales,ln_avg_hourly_earnings,ln_PCE_core,ind,pred_ln_cpi,pred_ln_cpi_goods,pred_ln_cpi_services,pred_ln_retail_sales,pred_ln_avg_hourly_earnings
2006-03-01,199.800,141.500,242.400,355665.0,20.05,83.509,5.297317,4.952300,5.490589,12.781745,2.998229,4.424954,0,5.327896,4.988944,5.472682,12.648661,3.008618
2006-04-01,201.500,141.700,243.200,357423.0,20.15,83.764,5.305789,4.953712,5.493884,12.786675,3.003204,4.428003,1,5.329219,4.988858,5.474835,12.651850,3.010573
2006-05-01,202.500,141.500,243.700,356704.0,20.13,83.974,5.310740,4.952300,5.495938,12.784662,3.002211,4.430507,2,5.330542,4.988773,5.476988,12.655039,3.012527
2006-06-01,202.900,140.700,244.700,357879.0,20.23,84.186,5.312713,4.946630,5.500033,12.787950,3.007167,4.433029,3,5.331865,4.988688,5.479140,12.658228,3.014482
2006-07-01,203.500,139.600,245.800,359006.0,20.29,84.268,5.315666,4.938781,5.504518,12.791094,3.010128,4.434002,4,5.333189,4.988602,5.481293,12.661417,3.016437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-01,307.026,168.029,399.219,699540.0,33.91,119.449,5.726932,5.124137,5.989510,13.458178,3.523710,4.782890,209,5.604411,4.971103,5.922603,13.315171,3.417147
2023-09-01,307.789,167.141,401.234,705304.0,34.01,119.842,5.729414,5.118838,5.994545,13.466384,3.526655,4.786174,210,5.605734,4.971018,5.924756,13.318360,3.419101
2023-10-01,307.671,166.759,402.549,703528.0,34.10,120.010,5.729031,5.116550,5.997817,13.463863,3.529297,4.787575,211,5.607057,4.970932,5.926908,13.321549,3.421056
2023-11-01,307.051,165.367,404.143,703336.0,34.23,120.088,5.727014,5.108167,6.001769,13.463590,3.533102,4.788225,212,5.608380,4.970847,5.929061,13.324738,3.423011


In [27]:
np.diff(dat.cpi,12)

array([ -124.18 ,    34.676,    18.089,   -58.167,   141.205,  -249.929,
         317.483,  -315.405,   283.146,  -286.493,   359.683,  -490.08 ,
         651.69 ,  -817.29 ,   937.737,  -966.05 ,   901.078,  -760.858,
         549.735,  -318.447,   208.084,  -318.677,   524.518,  -546.44 ,
         291.854,   -13.895,   -11.456,  -142.07 ,   193.311,   -34.744,
        -248.851,   581.284,  -886.945,  1019.779,  -886.725,   562.562,
        -190.095,  -154.986,   447.77 ,  -645.23 ,   699.559,  -613.964,
         453.185,  -295.616,   173.723,   -69.453,   -31.691,   108.898,
        -129.582,    75.124,    53.4  ,  -228.241,   393.589,  -494.526,
         519.679,  -493.151,   420.26 ,  -299.064,   189.877,  -185.091,
         300.469,  -464.84 ,   599.153,  -658.216,   623.924,  -492.375,
         275.132,   -12.329,  -233.221,   393.045,  -413.963,   314.13 ,
        -193.246,   133.216,  -145.153,   209.471,  -261.938,   204.634,
         -49.347,   -54.111,    22.495,    45.798, 